# Análisis y Visualización del Dataset de Detección de Fraude con Tarjetas de Crédito

**Dataset:** Credit Card Fraud Detection Dataset (Kaggle)

- Registros: 284,807
- Fraudes: 492 (0.172%)
- Atributos: Time, Amount, V1-V28 (PCA), Class

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configurar tamaño de figuras
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Cargar el dataset
print("Cargando dataset...")
df = pd.read_csv('datos/creditcard.csv')
print("Dataset cargado exitosamente!")
print(f"Forma del dataset: {df.shape}")

In [ ]:
# Información general del dataset
print("=" * 60)
print("INFORMACIÓN GENERAL DEL DATASET")
print("=" * 60)
df.info()

In [ ]:
# Primeras filas del dataset
print("\nPrimeras 5 filas del dataset:")
df.head()

In [ ]:
# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
df.describe()

In [ ]:
# Análisis de la variable objetivo (Class)
print("=" * 60)
print("ANÁLISIS DE LA VARIABLE OBJETIVO (Class)")
print("=" * 60)

class_counts = df['Class'].value_counts()
class_percentages = df['Class'].value_counts(normalize=True) * 100

print(f"\nTransacciones legítimas (0): {class_counts[0]:,} ({class_percentages[0]:.3f}%)")
print(f"Transacciones fraudulentas (1): {class_counts[1]:,} ({class_percentages[1]:.3f}%)")
print(f"\nRatio de desbalance: {class_counts[0]/class_counts[1]:.2f}:1")

## 1. Visualización de la Distribución de Clases

In [ ]:
# Gráfico de distribución de clases
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de barras
colors = ['#2ecc71', '#e74c3c']
axes[0].bar(['Legítima (0)', 'Fraude (1)'], class_counts.values, color=colors, alpha=0.7, edgecolor='black')
axes[0].set_ylabel('Número de Transacciones', fontsize=12, fontweight='bold')
axes[0].set_title('Distribución de Clases - Conteo', fontsize=14, fontweight='bold')
axes[0].set_yscale('log')
for i, v in enumerate(class_counts.values):
    axes[0].text(i, v, f'{v:,}', ha='center', va='bottom', fontweight='bold')

# Gráfico de pastel
axes[1].pie(class_counts.values, labels=['Legítima (0)', 'Fraude (1)'], 
            autopct='%1.3f%%', colors=colors, startangle=90, explode=(0, 0.1))
axes[1].set_title('Distribución de Clases - Porcentaje', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ Dataset altamente desbalanceado: Solo el 0.172% son fraudes")

## 2. Análisis de la Variable 'Time'

In [ ]:
# Convertir Time a horas
df['Time_hours'] = df['Time'] / 3600

print("=" * 60)
print("ANÁLISIS DE LA VARIABLE TIME")
print("=" * 60)
print(f"\nTiempo mínimo: {df['Time'].min()} segundos (0 horas)")
print(f"Tiempo máximo: {df['Time'].max()} segundos ({df['Time_hours'].max():.2f} horas)")
print(f"Duración total: {df['Time_hours'].max():.2f} horas ({df['Time_hours'].max()/24:.2f} días)")

In [ ]:
# Visualización de Time
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Distribución general de transacciones por tiempo
axes[0, 0].hist(df['Time_hours'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Tiempo (horas)', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Número de Transacciones', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Distribución de Transacciones en el Tiempo', fontsize=12, fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# Distribución por clase
legitimate = df[df['Class'] == 0]['Time_hours']
fraud = df[df['Class'] == 1]['Time_hours']

axes[0, 1].hist([legitimate, fraud], bins=50, label=['Legítima', 'Fraude'], 
                color=['#2ecc71', '#e74c3c'], alpha=0.6, edgecolor='black')
axes[0, 1].set_xlabel('Tiempo (horas)', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Número de Transacciones', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Distribución Temporal por Clase', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Boxplot de Time por clase
df.boxplot(column='Time_hours', by='Class', ax=axes[1, 0], patch_artist=True)
axes[1, 0].set_xlabel('Clase (0=Legítima, 1=Fraude)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Tiempo (horas)', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Distribución de Time por Clase', fontsize=12, fontweight='bold')
plt.sca(axes[1, 0])
plt.xticks([1, 2], ['Legítima (0)', 'Fraude (1)'])

# Densidad de fraudes vs tiempo
axes[1, 1].hist(fraud, bins=48, color='#e74c3c', alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Tiempo (horas)', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Número de Fraudes', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Distribución Temporal de Fraudes', fontsize=12, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Análisis de la Variable 'Amount'

In [ ]:
print("=" * 60)
print("ANÁLISIS DE LA VARIABLE AMOUNT")
print("=" * 60)

print("\nEstadísticas globales:")
print(f"Monto mínimo: ${df['Amount'].min():.2f}")
print(f"Monto máximo: ${df['Amount'].max():.2f}")
print(f"Monto promedio: ${df['Amount'].mean():.2f}")
print(f"Monto mediano: ${df['Amount'].median():.2f}")
print(f"Desviación estándar: ${df['Amount'].std():.2f}")

print("\nEstadísticas por clase:")
print("\nTransacciones Legítimas:")
print(f"Monto promedio: ${df[df['Class']==0]['Amount'].mean():.2f}")
print(f"Monto mediano: ${df[df['Class']==0]['Amount'].median():.2f}")

print("\nTransacciones Fraudulentas:")
print(f"Monto promedio: ${df[df['Class']==1]['Amount'].mean():.2f}")
print(f"Monto mediano: ${df[df['Class']==1]['Amount'].median():.2f}")

In [ ]:
# Visualización de Amount
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Histograma general (escala log)
axes[0, 0].hist(df['Amount'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Monto ($)', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Frecuencia', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Distribución del Monto de Transacciones', fontsize=12, fontweight='bold')
axes[0, 0].set_yscale('log')
axes[0, 0].grid(alpha=0.3)

# Comparación por clase
legitimate_amount = df[df['Class'] == 0]['Amount']
fraud_amount = df[df['Class'] == 1]['Amount']

axes[0, 1].hist([legitimate_amount, fraud_amount], bins=50, 
                label=['Legítima', 'Fraude'], 
                color=['#2ecc71', '#e74c3c'], alpha=0.6, edgecolor='black')
axes[0, 1].set_xlabel('Monto ($)', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Frecuencia', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Distribución de Montos por Clase', fontsize=12, fontweight='bold')
axes[0, 1].set_yscale('log')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Boxplot por clase
df.boxplot(column='Amount', by='Class', ax=axes[1, 0], patch_artist=True)
axes[1, 0].set_xlabel('Clase (0=Legítima, 1=Fraude)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Monto ($)', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Distribución de Montos por Clase (Boxplot)', fontsize=12, fontweight='bold')
axes[1, 0].set_yscale('log')
plt.sca(axes[1, 0])
plt.xticks([1, 2], ['Legítima (0)', 'Fraude (1)'])

# Violin plot
parts = axes[1, 1].violinplot([legitimate_amount[legitimate_amount > 0], 
                                fraud_amount[fraud_amount > 0]], 
                               positions=[0, 1], showmeans=True, showmedians=True)
axes[1, 1].set_xlabel('Clase', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Monto ($)', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Distribución de Montos por Clase (Violin Plot)', fontsize=12, fontweight='bold')
axes[1, 1].set_xticks([0, 1])
axes[1, 1].set_xticklabels(['Legítima (0)', 'Fraude (1)'])
axes[1, 1].set_yscale('log')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Análisis de las Variables PCA (V1-V28)

In [ ]:
# Seleccionar variables V1-V28
v_columns = [col for col in df.columns if col.startswith('V')]
print(f"Variables PCA encontradas: {len(v_columns)}")
print(f"Variables: {', '.join(v_columns)}")

In [ ]:
# Matriz de correlación de las variables PCA
plt.figure(figsize=(16, 14))
correlation_matrix = df[v_columns].corr()
sns.heatmap(correlation_matrix, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación de Variables PCA (V1-V28)', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Distribución de algunas variables PCA clave
fig, axes = plt.subplots(3, 3, figsize=(18, 14))
axes = axes.ravel()

# Seleccionar las primeras 9 variables V
for i, col in enumerate(v_columns[:9]):
    legitimate_v = df[df['Class'] == 0][col]
    fraud_v = df[df['Class'] == 1][col]
    
    axes[i].hist([legitimate_v, fraud_v], bins=50, 
                 label=['Legítima', 'Fraude'],
                 color=['#2ecc71', '#e74c3c'], alpha=0.6, edgecolor='black')
    axes[i].set_xlabel(col, fontsize=10, fontweight='bold')
    axes[i].set_ylabel('Frecuencia', fontsize=10, fontweight='bold')
    axes[i].set_title(f'Distribución de {col}', fontsize=11, fontweight='bold')
    axes[i].legend(fontsize=8)
    axes[i].grid(alpha=0.3)

plt.suptitle('Distribución de Variables PCA (V1-V9) por Clase', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

## 5. Análisis de Correlación con la Variable Objetivo

In [ ]:
# Calcular correlación de todas las variables con Class
correlations = df.corr()['Class'].drop('Class').sort_values(ascending=False)

print("=" * 60)
print("CORRELACIÓN CON LA VARIABLE OBJETIVO (Class)")
print("=" * 60)
print("\nTop 10 variables con mayor correlación positiva:")
print(correlations.head(10))
print("\nTop 10 variables con mayor correlación negativa:")
print(correlations.tail(10))

In [ ]:
# Visualizar correlaciones
plt.figure(figsize=(12, 8))
correlations.plot(kind='barh', color=['#e74c3c' if x < 0 else '#2ecc71' for x in correlations])
plt.xlabel('Correlación con Class', fontsize=12, fontweight='bold')
plt.ylabel('Variables', fontsize=12, fontweight='bold')
plt.title('Correlación de Variables con la Variable Objetivo (Class)', 
          fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 6. Análisis de Variables más Relevantes para Fraude

In [ ]:
# Obtener las 6 variables más correlacionadas (positiva y negativamente)
top_positive = correlations.head(3).index.tolist()
top_negative = correlations.tail(3).index.tolist()
top_features = top_positive + top_negative

print(f"Variables más relevantes para detectar fraude:")
print(f"\nCorrelación positiva: {', '.join(top_positive)}")
print(f"Correlación negativa: {', '.join(top_negative)}")

In [ ]:
# Visualizar las variables más importantes
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, feature in enumerate(top_features):
    legitimate_f = df[df['Class'] == 0][feature]
    fraud_f = df[df['Class'] == 1][feature]
    
    axes[i].hist([legitimate_f, fraud_f], bins=50,
                 label=['Legítima', 'Fraude'],
                 color=['#2ecc71', '#e74c3c'], alpha=0.6, edgecolor='black')
    axes[i].set_xlabel(feature, fontsize=11, fontweight='bold')
    axes[i].set_ylabel('Frecuencia', fontsize=11, fontweight='bold')
    axes[i].set_title(f'{feature} (Corr: {correlations[feature]:.3f})', 
                      fontsize=12, fontweight='bold')
    axes[i].legend()
    axes[i].grid(alpha=0.3)
    axes[i].set_yscale('log')

plt.suptitle('Variables más Correlacionadas con Fraude', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

## 7. Scatter Plots de Variables Clave

In [ ]:
# Crear scatter plots de las variables más importantes
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Tomar una muestra para mejor visualización
sample_size = 5000
df_sample = df.sample(n=min(sample_size, len(df)), random_state=42)

# Plot 1: V17 vs V14
for class_val, color, label in [(0, '#2ecc71', 'Legítima'), (1, '#e74c3c', 'Fraude')]:
    mask = df_sample['Class'] == class_val
    axes[0].scatter(df_sample[mask]['V17'], df_sample[mask]['V14'],
                   c=color, alpha=0.5, s=30, label=label, edgecolors='black', linewidth=0.5)

axes[0].set_xlabel('V17', fontsize=12, fontweight='bold')
axes[0].set_ylabel('V14', fontsize=12, fontweight='bold')
axes[0].set_title('V17 vs V14', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: V12 vs V10
for class_val, color, label in [(0, '#2ecc71', 'Legítima'), (1, '#e74c3c', 'Fraude')]:
    mask = df_sample['Class'] == class_val
    axes[1].scatter(df_sample[mask]['V12'], df_sample[mask]['V10'],
                   c=color, alpha=0.5, s=30, label=label, edgecolors='black', linewidth=0.5)

axes[1].set_xlabel('V12', fontsize=12, fontweight='bold')
axes[1].set_ylabel('V10', fontsize=12, fontweight='bold')
axes[1].set_title('V12 vs V10', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.suptitle('Relación entre Variables PCA más Relevantes', 
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 8. Resumen Estadístico Final

In [ ]:
print("=" * 70)
print("RESUMEN ESTADÍSTICO DEL DATASET")
print("=" * 70)

print(f"\n📊 INFORMACIÓN GENERAL:")
print(f"   Total de transacciones: {len(df):,}")
print(f"   Total de atributos: {len(df.columns)}")
print(f"   Transacciones legítimas: {class_counts[0]:,} ({class_percentages[0]:.3f}%)")
print(f"   Transacciones fraudulentas: {class_counts[1]:,} ({class_percentages[1]:.3f}%)")
print(f"   Ratio de desbalance: {class_counts[0]/class_counts[1]:.2f}:1")

print(f"\n⏱️ TIEMPO:")
print(f"   Duración del dataset: {df['Time_hours'].max():.2f} horas ({df['Time_hours'].max()/24:.2f} días)")

print(f"\n💰 MONTOS:")
print(f"   Rango de montos: ${df['Amount'].min():.2f} - ${df['Amount'].max():.2f}")
print(f"   Monto promedio (legítimas): ${df[df['Class']==0]['Amount'].mean():.2f}")
print(f"   Monto promedio (fraudes): ${df[df['Class']==1]['Amount'].mean():.2f}")

print(f"\n🔍 VARIABLES PCA:")
print(f"   Número de componentes principales: {len(v_columns)}")
print(f"   Variable más correlacionada (+): {correlations.idxmax()} ({correlations.max():.3f})")
print(f"   Variable más correlacionada (-): {correlations.idxmin()} ({correlations.min():.3f})")

print(f"\n✅ CALIDAD DE DATOS:")
print(f"   Valores faltantes: {df.isnull().sum().sum()}")
print(f"   Duplicados: {df.duplicated().sum()}")

print("\n" + "=" * 70)

## 9. Exportar Resumen de Datos

In [ ]:
# Crear un resumen para exportar
resumen = {
    'Total_Transacciones': len(df),
    'Transacciones_Legitimas': class_counts[0],
    'Transacciones_Fraudulentas': class_counts[1],
    'Porcentaje_Fraude': class_percentages[1],
    'Duracion_Horas': df['Time_hours'].max(),
    'Monto_Promedio_Legitimas': df[df['Class']==0]['Amount'].mean(),
    'Monto_Promedio_Fraudes': df[df['Class']==1]['Amount'].mean(),
    'Ratio_Desbalance': class_counts[0]/class_counts[1]
}

resumen_df = pd.DataFrame([resumen])
resumen_df.to_csv('resumen_dataset_fraude.csv', index=False)
print("✅ Resumen exportado a 'resumen_dataset_fraude.csv'")
resumen_df

# Análisis de Fraude de Tarjetas de Crédito

Este cuaderno analiza el conjunto de datos de detección de fraude de tarjetas de crédito de Kaggle.

**Conjunto de datos:**
*   **Nombre original:** Credit Card Fraud Detection Dataset
*   **Fuente:** Kaggle
*   **Número de registros:** 284,807
*   **Número de fraudes:** 492 (0.172%)
*   **Atributos:**
    *   `Time`, `Amount`
    *   `V1` – `V28`: Componentes principales generadas por PCA
    *   `Class`: Variable objetivo (0 = legítima, 1 = fraude)

## 1. Importar Librerías

Importar las librerías necesarias para el análisis y visualización de datos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para los gráficos
sns.set_style('whitegrid')
%matplotlib inline

## 2. Cargar y Explorar el Conjunto de Datos

Cargamos el archivo `creditcard.csv` en un DataFrame de pandas. Es importante que el archivo esté en la carpeta `datos`. Luego, mostraremos información básica del DataFrame.

In [ ]:
# Cargar el dataset
# Asegúrate de que el archivo creditcard.csv esté en la carpeta 'datos'
try:
    df = pd.read_csv('datos/creditcard.csv')
    print("Archivo cargado exitosamente.")
except FileNotFoundError:
    print("Error: No se encontró el archivo 'datos/creditcard.csv'.")
    print("Por favor, descarga el conjunto de datos de Kaggle y colócalo en la carpeta 'datos'.")

# Si el archivo se cargó, mostrar las primeras 5 filas
if 'df' in locals():
    display(df.head())

In [ ]:
# Mostrar información general y estadísticas descriptivas
if 'df' in locals():
    print("Información del DataFrame:")
    df.info()
    print("\nEstadísticas Descriptivas:")
    display(df.describe())

## 3. Visualizar la Distribución de Clases

Visualizamos la distribución de transacciones legítimas y fraudulentas para observar el desequilibrio en los datos.

In [ ]:
if 'df' in locals():
    plt.figure(figsize=(8, 6))
    sns.countplot(x='Class', data=df)
    plt.title('Distribución de Clases (0: Legítima, 1: Fraude)')
    plt.show()

    # Contar el número de fraudes y transacciones legítimas
    class_counts = df['Class'].value_counts()
    print(class_counts)
    print(f"Porcentaje de fraude: {class_counts[1] / class_counts.sum() * 100:.4f}%")

## 4. Analizar las Características 'Time' y 'Amount'

Analizamos la distribución de las características `Time` y `Amount`.

In [ ]:
if 'df' in locals():
    fig, ax = plt.subplots(1, 2, figsize=(18, 4))

    sns.histplot(df['Amount'], ax=ax[0], color='r', bins=50)
    ax[0].set_title('Distribución de Amount')

    sns.histplot(df['Time'], ax=ax[1], color='b', bins=50)
    ax[1].set_title('Distribución de Time')

    plt.show()

## 5. Visualizar la Correlación entre Características

Generamos un mapa de calor para ver la correlación entre las características. Dado que las características V son resultado de PCA, no esperamos ver una fuerte correlación entre ellas, pero sí es interesante ver su correlación con `Amount`, `Time` y `Class`.